In [73]:
import sys
import re
import pymongo
import json
import time
import datetime
import requests
import numpy
from bs4 import BeautifulSoup

### Get GitLab Projects

In [66]:
dbname = "fdac18mp2" #please use this database
collname = "glprj_eezell3" #please modify so you store data in your collection
# beginning page index
begin = "0"
client = pymongo.MongoClient()

db = client[dbname]
coll = db[collname]


beginurl = "https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=" + begin + \
    "&per_page=99&simple=false&sort=desc&starred=false&statistics=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false"


gleft = 0

header = {'per_page': 99}

# check remaining query chances for rate-limit restriction
def wait(left):
    global header
    while (left < 20):
        l = requests.get('https://gitlab.com/api/v4/projects', headers=header)
        if (l.ok):
            left = int(l.headers.get('RateLimit-Remaining'))
        time .sleep(60)
    return left

# send queries and extract urls 
def get(url, coll):

    global gleft
    global header
    global bginnum
    gleft = wait(gleft)
    values = []
    size = 0

    try:
        r = requests .get(url, headers=header)
        
        time .sleep(0.5)
        # got blocked
        if r.status_code == 403:
            return "got blocked", str(bginnum)
        if (r.ok):

            gleft = int(r.headers.get('RateLimit-Remaining'))
            lll = r.headers.get('Link')
            t = r.text
            array = json.loads(t)
            
            for el in array:
                coll.insert_one(el)
 
            #next page
            while ('; rel="next"' in lll):
                gleft = int(r.headers.get('RateLimit-Remaining'))
                gleft = wait(gleft)
                # extract next page url
                ll = lll.replace(';', ',').split(',')
                url = ll[ll.index(' rel="next"') -
                         1].replace('<', '').replace('>', '').lstrip()
             
                try:
                    r = requests .get(url, headers=header)
                    if r.status_code == 403:
                        return "got blocked", str(bginnum)
                    if (r.ok):
                        lll = r.headers.get('Link')
                        t = r.text
                        array1 = json.loads(t)
                        for el in array1:
                            coll.insert_one(el)
                    else:
                        sys.stderr.write("url can not found:\n" + url + '\n')
                        return 
                except requests.exceptions.ConnectionError:
                    sys.stderr.write('could not get ' + url + '\n')

        else:
            sys.stderr.write("url can not found:\n" + url + '\n')
            return

    except requests.exceptions.ConnectionError:
        sys.stderr.write('could not get ' + url + '\n')
    except Exception as e:
        sys.stderr.write(url + ';' + str(e) + '\n')


In [67]:
#start retrieving GitLab projects      
get(beginurl,coll)

KeyboardInterrupt: 

### get SourceForge projects

In [49]:
# create set for matches
e_projs = set()

for page_num in range(1,50):
    r = requests .get("https://sourceforge.net/directory/os%3Amac/?q=e&page=" + str(page_num))

    # check if we made a valid request
    if r.status_code is not 200:
        break
    
    soup = BeautifulSoup(r.text, 'html.parser')
    
    for link in soup.find_all('a'):
        href = link.get('href')
        match = re.match('/projects/(e\w+/)',str(href))
        if match:
            e_projs.add(match.group(1))

In [68]:
dbname = "fdac18mp2" #please use this database
collname = "glprj_eezell3" #please modify so you store data in your collection

client = pymongo.MongoClient()

db = client[dbname]
coll = db[collname]

## get project data and insert into db
for project in e_projs:
    r = requests .get("https://sourceforge.net/rest/p/" + project)
    coll.insert_one(r.json())

DuplicateKeyError: E11000 duplicate key error collection: fdac18mp2.glprj_eezell3 index: _id_ dup key: { : "5082d82c0594ca30de719a5b" }

In [69]:
print(e_projs)

{'esencryption/', 'ergosum/', 'einsoft/', 'ephysics/', 'egovstack/', 'esercizicpp/', 'eibs/', 'ebreadboard/', 'enounce/', 'eodbo/', 'ericoptimize/', 'emailsender2015/', 'epluscms/', 'easyfoldermorpher/', 'elasticvectorspaceanalysis/', 'egrovetest/', 'einkdisplayforshoppingtags/', 'edonkeyplus/', 'easer/', 'educatux/', 'ebooknavigation/', 'eggsengine/', 'emailrelay/', 'easyrec/', 'ebyte/', 'edofolder/', 'electronicbrown/', 'ebookgratis/', 'edebian/', 'extpn/', 'epmanonlineproj/', 'easylogging827/', 'emgukalman/', 'emusorganizer/', 'eimza/', 'envmailgmaihot/', 'ematpro/', 'exponentcms/', 'esuite/', 'ekocalc/', 'euaeppcjit/', 'eteachingkit/', 'erosvitor/', 'ebooks1/', 'esvaasthya/', 'evotingplugin/', 'electronictheme/', 'estudy/', 'easyarticles/', 'efax/', 'ehalal/', 'easycbr/', 'estorea2z/', 'eexam/', 'emaqdot/', 'esolutionlive/', 'entrada/', 'extjs4erp/', 'emailsepador/', 'educate/', 'ekoconv/', 'edifact/', 'encrypto/', 'eproofelectron/', 'emarkettdsoft/', 'emu2utils/', 'elearnvk/', 'ec